In [ ]:
import xarray as xr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set_style('darkgrid')

# Getting the scalar so cumulative emissions are from 1750

In [ ]:
rcp_emissions= xr.open_dataset(
    "/shares/gcp/integration/float32/dscim_input_data/climate/AR6/ar6_fair162_control_pulse_2020-2030-2040-2050-2060-2070-2080_emis_conc_rf_temp_lambdaeff_emissions-driven_naturalfix_v4.0_Jan212022.nc"
).sel(gas='CO2_Fossil', runtype='control', pulse_year=2020, drop=True)

scale_1850 = rcp_emissions.emissions.sel(rcp='ssp245').sel(year=slice(1750,2019)).sum('year').item()

# RFF emissions trajectories

In [ ]:
drop_2300 = False
cumulative = True
save_path = '/mnt/CIL_integration/plots/rff_diagnostics/'

In [ ]:
emissions = xr.open_dataset(
    "/shares/gcp/integration/rff2/climate/emissions/rff-sp_emissions_all_gases.nc"
).sel(gas='C', drop=True)

if cumulative == True:
    summed = emissions.cumsum('Year').assign_coords({'Year' : emissions.Year}) + scale_1850
    plot = summed.idxmax('Year').emissions.rename('max_year').to_dataframe().reset_index()
    emission_type = 'Cumulative'
    lim=[0,2,70,75]
else:
    plot = emissions.idxmax('Year').emissions.rename('max_year').to_dataframe().reset_index()
    emission_type = 'Annual'
    lim=[0,15,15,30]

if drop_2300==True:
    plot = plot.loc[plot.max_year != 2300]

In [ ]:
plot.to_csv(f"{save_path}/brookings_data/rff_histogram_cumulative-{cumulative}_drop_2300-{drop_2300}.csv")

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, figsize=(15,8))

for a in ax1, ax2:
    ax = sns.histplot(
        data=plot,
        x='max_year',
        stat='percent',
        bins=100,
        ax=a
    )
    
ax2.set_ylim(lim[0], lim[1])
ax1.set_ylim(lim[2], lim[3])
ax1.set_ylabel('')
ax2.set_ylabel('')

ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

ax1.axes.xaxis.set_ticks([])
ax1.set_xlabel('')
ax2.set_xlabel('Year of maximum')

ax1.spines['top'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['right'].set_visible(False)

# plt.annotate(
#     f"{round(plot.max_year.describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]),2)}", 
#     (0.5, 0.5), 
#     xycoords='axes fraction')

plt.suptitle(f"Histogram: Year of Maximum {emission_type} Emissions, RFF GtC", fontsize=18)

if cumulative == True:
    plt.subplots_adjust(hspace=0.1, top=0.95)
    d = .005  # how big to make the diagonal lines in axes coordinates
    # arguments to pass to plot, just so we don't keep repeating them
    kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
    ax1.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal

    kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
else:
    plt.subplots_adjust(hspace=0, top=0.95)

plt.annotate("Percent", (0.04, 0.5), rotation=90, xycoords='figure fraction')

plt.savefig(f'{save_path}/max_year_{emission_type}_emissions_drop2300-{drop_2300}_broken_axis.pdf')

In [ ]:
summy = 0 
for ranges in [range(2001+i*50, 2001+(i+1)*50) for i in range(6)]:

    print('Year', ranges, ':', plot.loc[plot.max_year.isin(ranges)].max_year.count() / len(plot.max_year) * 100, '%')
    
    summy+=plot.loc[plot.max_year.isin(ranges)].max_year.count() / len(plot.max_year) * 100
    
assert round(summy,5) == 100

# Heatmap of all RFF simulations annual emissions

In [ ]:
listy = []
# bins = np.linspace(-15, 40, 12)
bins = np.arange(-13, 41, 1)
interval = 1

for y in np.arange(2020,2300, interval):
    
    if interval==10:
        slicey=slice(y,y+9)
    elif interval==1:
        slicey=slice(y,y)

    sub = emissions.sel(Year=slicey, drop=True).groupby_bins('emissions', bins).count().to_dataframe().reset_index()
    sub['Year'] = y

    listy.append(sub)

In [ ]:
frame = pd.concat(listy)
frame['Annual emissions'] = frame['index'].apply(lambda x : float(x.left))
frame['percent'] = frame.emissions/10000*100
data= frame.pivot('Annual emissions', 'Year', 'percent').sort_index(ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))

sns.heatmap(
    data=data,
    ax=ax,
    cmap='magma_r'
)

ax.set_xlabel('Year')
fig.suptitle("Density plot of annual emissions (GtC)")

ax.set_xticks(ax.get_xticks()[::5])
ax.set_yticks(ax.get_yticks()[::5])

plt.xticks(rotation=360) 
plt.yticks(rotation=360) 

plt.savefig(f'{save_path}/annual_emissions_heatmap_single_year_single_emission.pdf', bbox_inches='tight')

# How many years have the same 2300 cumulative emissions value OR same max value

In [ ]:
if cumulative == True:
    # year_type = 'max_year'
    year_type = 'share_2300_year'
    
    if year_type=='max_year':
        find_item = summed.max('Year')
    elif year_type == 'share_2300_year':
        find_item = summed.sel(Year=2300, drop=True)
        
    years = summed.where(
        np.round(summed,4) == np.round(find_item,4), 
        drop=True
    ).to_dataframe()

    unique_years = years.loc[~years.emissions.isnull()].groupby('simulation').count()

    non_unique_simulations = list(unique_years.loc[unique_years.emissions > 1].index)
    
    fig, ax = plt.subplots(figsize=(20,8))

    sns.lineplot(
        data=summed.sel(simulation=non_unique_simulations).to_dataframe().reset_index(),
        x='Year',
        y='emissions',
        hue='simulation',
        ax=ax
    )

    plt.suptitle(f"Trajectories of non-unique {year_type} simulations ({len(non_unique_simulations)} simulations)")

    plt.savefig(f'{save_path}/non_unique_{year_type}_simulations.png', dpi=300, bbox_inches='tight')

# RCP emissions trajectories

In [ ]:
rcp_emissions= xr.open_dataset(
    "/shares/gcp/integration/float32/dscim_input_data/climate/AR6/ar6_fair162_control_pulse_2020-2030-2040-2050-2060-2070-2080_emis_conc_rf_temp_lambdaeff_emissions-driven_naturalfix_v4.0_Jan212022.nc"
).sel(gas='CO2_Fossil', runtype='control', pulse_year=2020, year=slice(2020,2300))

summed_rcp = (rcp_emissions.cumsum('year').assign_coords({'year' : rcp_emissions.year}) + scale_1850).emissions

In [ ]:
summed_rcp.to_dataframe().to_csv(f"{save_path}/brookings_data/rcp_histogram_cumulative-True_drop_2300-{drop_2300}.csv")

In [ ]:
second_deriv = summed_rcp.differentiate('year')
second_deriv.where(second_deriv==0, drop=True).idxmin('year').to_dataframe()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

sns.lineplot(
    data=summed_rcp.to_dataframe().reset_index(),
    x='year',
    y='emissions',
    hue='rcp',
    ax=ax
)

fig.suptitle("Cumulative Emissions for each RCP, GtC")

plt.savefig(f'{save_path}/rcp_cum_emissions.png', dpi=300, bbox_inches='tight')

# Combine RFF and RCP graphs

In [ ]:
quantiles=[(0.01, 0.99), (0.05, 0.95)]

if cumulative==True:
    rff_quantiles = summed
    rcp = summed_rcp
else:
    rff_quantiles = emissions.emissions 
    rcp = rcp_emissions.emissions 

rff_quantiles = rff_quantiles.quantile(
    [i[0] for i in quantiles] + [i[1] for i in quantiles] + [0.5], 
    'simulation'
).to_dataframe().reset_index()

rcp = rcp.to_dataframe().reset_index()

In [ ]:
rff_quantiles.to_csv(f"{save_path}/brookings_data/rff_timeseries_cumulative-{cumulative}_drop_2300-{drop_2300}.csv")
rcp.to_csv(f"{save_path}/brookings_data/rcp_timeseries_cumulative-{cumulative}_drop_2300-{drop_2300}.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))

for col, q in enumerate(quantiles):

    # ribbon plots
    ax.fill_between(x=rff_quantiles.loc[rff_quantiles['quantile'] == q[0]].Year, 
                     y1=rff_quantiles.loc[rff_quantiles['quantile'] == q[0]].emissions, 
                     y2=rff_quantiles.loc[rff_quantiles['quantile'] == q[1]].emissions,
                     color=sns.color_palette("Greys", len(quantiles))[col], 
                     alpha=0.3,
                     label=f"RFF quantile : {q}",
                    )
    
# RFF median
sns.lineplot(data=rff_quantiles.loc[rff_quantiles['quantile'] == 0.5],
             x='Year',
             y='emissions',
             ax=ax,
             color='black',
             label="RFF quantile: 0.5",
            )

# rcp scenarios
palette = sns.color_palette("husl", len(rcp.rcp.unique()))
sns.lineplot(data=rcp,
             x='year',
             y='emissions',
             ax=ax,
             hue='rcp',
             palette=palette
            )

ax.legend(bbox_to_anchor=(0.65,-0.1), ncol=2)

ax.set_ylabel('Emissions')

fig.suptitle(f"RFF and RCP {emission_type} Emissions, GtC")

plt.savefig(f'{save_path}/rff_rcp_{emission_type}_emissions.pdf', bbox_inches='tight')